# CHAOS Liver Segmentation (CT) — U-Net (Clean Notebook)

Ce notebook contient une version **propre** et **lisible** du pipeline de segmentation du foie sur le dataset **CHAOS (CT)**.

**Résumé :**
- Prétraitement : DICOM → HU → fenêtre *foie* fixe *(center=60, width=150)* → normalisation [0,1] → resize 256×256.
- Masques : binaires (0/1), resize **NEAREST**.
- Split : **par patient** (80/20).
- Modèle : **U-Net** (Keras/TensorFlow 2.10), **loss = BCE + SoftDice**.
- Évaluation : balayage de seuil (0.1–0.6) + **Largest Connected Component**.
- Résultats val typiques : **Dice ≈ 0.71**, **IoU ≈ 0.68**.

> ℹ️ L'appel d'entraînement `model.fit(...)` est **commenté** pour éviter un run accidentel (~6h). Décommentez si vous souhaitez réentraîner.


## 0) Préambule & chemins
**Cellule (à écrire)** — imports, chemins, constantes, puis vérifications.

In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pydicom
import cv2
import tensorflow as tf
import random

# dézipper
import zipfile, os
zip_name = 'CHAOS_Train_Sets.zip'
if os.path.exists(zip_name):
            zf.extractall('.') # extrait tous les fichiers du ZIP dans le dossier courant ('.').
            print('Extrait:', zip_name)
            
CT_ROOT = Path('Train_Sets/CT')

TARGET_SIZE = 256  # multiple de 16
USE_WINDOW = True
DEFAULT_CW = (60.0, 150.0)  # (center, width) CT abdomen
BATCH_SIZE = 4
SEED = 42

# ✅ Check attendu (à lancer quand variables prêtes)
print(CT_ROOT.exists())  # → True
print(tf.__version__)


## 1) Lister les patients & en sélectionner quelques‑uns

**Cellule — Lister**

In [ ]:
patients = sorted(patients, key=lambda p: int(p.name))
print([p.name for p in patients])


**Cellule — Sélection**

In [ ]:
N_PAT= 15 # choisi 15 patients sur 20
import random; random.seed(SEED)
sel_patients = sorted(random.sample(patients, N_PAT), key=lambda p: int(p.name))

# ✅ Check attendu
print([int(p.name) for p in sel_patients])


## 2) Appairage **DICOM ↔ masque** par patient (filtrage masques vides)

**Cellule — : écrire `pairs_for_one_patient(p_dir)`**

In [ ]:
def pairs_for_one_patient(p_dir, pos_to_neg_ratio=3, seed=1337): #     Crée les paires (DICOM, masque, id_patient). Garde toutes les tranches POSITIVES (avec foie). Sous-échantillonne les tranches NEGATIVES pour viser ~pos:neg ≈ 3:1.
# Définit les chemins des deux sous-dossiers.
    dcm_dir = p_dir / "DICOM_anon"
    msk_dir = p_dir / "Ground"
# Liste tous les fichiers .dcm et .png, puis trie (ordre alphabétique → correspondance par index).
    dcms  = sorted(dcm_dir.glob("*.dcm"))
    masks = sorted(msk_dir.glob("*.png"))
    n = min(len(dcms), len(masks)) # prend min image entre les deux 
    
    pos, neg = [], []
    for i in range(n):
        d_path, m_path = dcms[i], masks[i]
        m = np.array(Image.open(m_path).convert("L"))
        if m.max() > 0:   # tranche POSITIVE (au moins un pixel foie)
            pos.append((d_path, m_path, int(p_dir.name)))
        else:             # tranche NEGATIVE
            neg.append((d_path, m_path, int(p_dir.name)))

    # Sous-échantillonnage des négatives pour ratio ≈ 3:1
    rng = np.random.default_rng(seed)
    if len(pos) and len(neg):
        max_neg = max(1, int(np.ceil(len(pos) / pos_to_neg_ratio)))
        if len(neg) > max_neg:
            neg = list(rng.choice(neg, size=max_neg, replace=False))

    # Retourne toutes les positives + sous-échantillon de négatives
    pairs = pos + neg
    rng.shuffle(pairs)
    return pairs


**Cellule — Appliquer multi‑patients**

In [ ]:
all_pairs = []
for pdir in sel_patients:
    all_pairs.extend(pairs_for_one_patient(pdir, pos_to_neg_ratio=2))

# ✅ Check attendu
print('Total paires:', len(all_pairs))
print(all_pairs[0])  # → (Path(...dcm), Path(...png), pid)


In [ ]:
pos = sum(np.array(Image.open(m).convert("L")).max()>0 for _,m,_ in all_pairs)
neg = len(all_pairs)-pos
print("pos/neg =", pos, "/", neg)


## 3) Split **par patient** (pas par slice)

In [ ]:
unique_pat_ids = sorted({pid for _,_,pid in all_pairs}) # for _,_,pid in all_pairs → on parcourt toutes les paires (DICOM, masque, patient_id). On ne garde que pid.
n_train = int(0.8 * len(unique_pat_ids)) # 80% test
train_ids = set(unique_pat_ids[:n_train]) # train = premier 80%
val_ids   = set(unique_pat_ids[n_train:]) # validation= dernier 20%

pairs_train = [(d,m) for (d,m,pid) in all_pairs if pid in train_ids] # Si l’ID patient est dans train_ids → on envoie la paire dans pairs_train.
pairs_val   = [(d,m) for (d,m,pid) in all_pairs if pid in val_ids] # idem validation

# ✅ Check attendu
print('Patients train:', sorted(train_ids))
print('Patients val  :', sorted(val_ids))
print('Slices train/val:', len(pairs_train), len(pairs_val))


## 4) Fonctions de **prétraitement**

In [ ]:

import pydicom
import cv2
import numpy as np


def apply_rescale(ds, arr):
    slope = float(getattr(ds, 'RescaleSlope', 1.0))
    intercept = float(getattr(ds, 'RescaleIntercept', 0.0))
    return arr * slope + intercept

def apply_window(ds, arr, default_cw=(60.0, 150.0)):
    # CHANGEMENT: on n’utilise plus ds.WindowCenter/WindowWidth.
    #             On force une fenêtre foie FIXE pour toutes les tranches/patients.

    # CHANGEMENT: définir directement c,w = 60,150 (ou default_cw si tu veux garder un paramètre)
    c, w = default_cw                     # CHANGEMENT: fenêtrage foie fixe

    lo, hi = c - w/2.0, c + w/2.0         # bords de fenêtre
    arr = np.clip(arr, lo, hi)            # clip dans [lo, hi]
    arr = (arr - lo) / (hi - lo + 1e-6)   # map linéairement vers [0,1]
    return arr

def normalize01(arr):
    # CHANGEMENT: on NE fait plus de min–max par tranche pour le CT.
    # Si 'arr' sort de apply_window(...), il est déjà dans [0,1].
    # On se contente d’assurer le dtype et de clipper au cas où.
    arr = arr.astype(np.float32)
    return np.clip(arr, 0.0, 1.0)   # CHANGEMENT: no-op stable (aucun recalage par image)

def to_network_size(arr, target_size=256):
    arr = cv2.resize(arr, (target_size, target_size), interpolation=cv2.INTER_AREA)
    # CHANGEMENT: ne pas renormaliser; on reste dans [0,1] grâce à apply_window fixe
    arr = np.clip(arr, 0.0, 1.0)        # sécurité légère
    arr = arr[..., None].astype("float32")
    return arr

def load_dicom_preprocessed(path, target_size=256, use_window=True):
    ds  = pydicom.dcmread(str(path))                    # <-- utiliser 'path'
    img = ds.pixel_array.astype(np.float32)

    # 1) Rescale (HU pour CT)
    img = apply_rescale(ds, img)
    
    # 2) Fenêtrage FOIE FIXE -> [0,1]
    # CHANGEMENT: même si use_window=False, on applique quand même la fenêtre fixe
    # pour éviter TOUT min–max par tranche.
    if use_window:
        img = apply_window(ds, img)     # maintenant: (60,150) fixe
    else:
        img = apply_window(ds, img)     # CHANGEMENT: plus de normalize01 ici



    # 3) Resize + (H,W,1) float32
    img = to_network_size(img, target_size)
    return img


In [ ]:
# Pick un DICOM de ton dataset (adapter le chemin)
some_dicom_path = "Train_Sets/CT/1/DICOM_anon/i0007,0000b.dcm"

x = load_dicom_preprocessed(some_dicom_path)
print("Shape:", x.shape)        # attendu: (256,256,1)
print("Dtype:", x.dtype)        # attendu: float32
print("Min/Max:", float(x.min()), float(x.max()))  # attendu: ~0.0, ~1.0

# Visual check
import matplotlib.pyplot as plt
plt.imshow(x[...,0], cmap='gray')
plt.title("Window foie 60/150")
plt.axis('off')
plt.show()


**Cellule — : `load_mask_binary(path)`**

In [ ]:
# arr = np.array(Image.open(path).convert('L'))
# mask = (arr > 0).astype(np.uint8)
# # resize NEAREST vers TARGET_SIZE
# # retourner mask[..., None].astype('float32')
import cv2
import imageio.v2 as iio
import numpy as np

def load_mask_binary(path_png, target_size=256):
    m = iio.imread(path_png)  # lit le PNG
    if m.ndim == 3:           # si RGB → convertis en gris
        m = cv2.cvtColor(m, cv2.COLOR_RGB2GRAY)
    # binaire direct {0,1}
    m = (m > 0).astype(np.uint8)
    # resize en NEAREST
    m = cv2.resize(m, (target_size, target_size), interpolation=cv2.INTER_NEAREST)
    # (H,W,1) float32
    m = m[..., None].astype(np.float32)
    return m


**Check attendu — vérif overlay**

In [ ]:
d_path, m_path, pid = all_pairs[3]
img = load_dicom_preprocessed(d_path)
msk = load_mask_binary(m_path)
plt.figure(figsize=(9,3))
plt.subplot(1,3,1); plt.imshow(img[...,0], cmap='gray'); plt.title('Image'); plt.axis('off')
plt.subplot(1,3,2); plt.imshow(msk[...,0], cmap='gray'); plt.title('Mask GT'); plt.axis('off')
plt.subplot(1,3,3); plt.imshow(img[...,0], cmap='gray'); plt.imshow(msk[...,0],cmap="Reds", alpha=0.35); plt.title('Overlay'); plt.axis('off')
plt.show()


## 5) Construire `tf.data.Dataset` (train & val)

**Cellule — : listes de chemins**

In [ ]:
x_train = [str(d) for (d, m) in pairs_train]
y_train = [str(m) for (d, m) in pairs_train]

x_val   = [str(d) for (d, m) in pairs_val]
y_val   = [str(m) for (d, m) in pairs_val]

print(len(x_train), len(y_train), len(x_val), len(y_val))
assert len(x_train) == len(y_train)
assert len(x_val) == len(y_val)
print("ex:", x_train[0], "->", y_train[0])


**Cellule — : wrappers**

In [ ]:
#  - décoder bytes → str
#  - X = load_dicom_preprocessed(...); Y = load_mask_binary(...)
#  - return X, Y

# ========= 1) Loader Python (retourne des np.array) =========================
def py_load_pair(path_dcm, path_png):
    # tf.py_function passe des bytes -> on reconvertit en str Python
    path_dcm = path_dcm.numpy().decode("utf-8") # on extrait le chemin DICOM de chaque paire, et on le met en str (TF préfère str/bytes).
    path_png = path_png.numpy().decode("utf-8")
    
    X= load_dicom_preprocessed(path_dcm)
    Y = load_mask_binary(path_png)

    return X, Y  # np.ndarray (T,T,1), np.ndarray (T,T,1)



#  - tf.py_function(py_load_pair, ..., Tout=[tf.float32, tf.float32])
#  - X.set_shape([TARGET_SIZE, TARGET_SIZE, 1]); Y.set_shape(...)
#  - return X, Y

# ========= 2) Wrapper TensorFlow autour du loader Python ====================
def tf_load_pair(path_dcm, path_png):
    # Appelle py_load_pair côté Python depuis le graphe TF
    X, Y = tf.py_function(
        func=py_load_pair,                      # fonction Python à appeler
        inp=[path_dcm, path_png],               # ses arguments (tensors -> bytes)
        Tout=[tf.float32, tf.float32],          # types de sortie attendus
    )
    # IMPORTANT: fixer la shape statique pour que TF connaisse la taille
    X.set_shape((TARGET_SIZE, TARGET_SIZE, 1))
    Y.set_shape((TARGET_SIZE, TARGET_SIZE, 1))
    
    return X, Y

# ========= 3) Data augmentation ====================
def augment(img, mask):
    # flip horizontal
    if tf.random.uniform(()) > 0.5: # tf.random.uniform(()) génère un scalaire uniforme dans [0,1). > 0.5 ⇒ 50% de chances de rentrer dans le bloc.  On décide au hasard si on applique ou non le flip horizontal.
    #On retourne l’image et le masque de gauche à droite. ⚠️ Très important : on applique exactement la même transfo aux deux, pour garder l’alignement.
        img  = tf.image.flip_left_right(img)
        mask = tf.image.flip_left_right(mask)
    # flip vertical
    if tf.random.uniform(()) > 0.5:
        img  = tf.image.flip_up_down(img)
        mask = tf.image.flip_up_down(mask)
    # rotation 90°
    if tf.random.uniform(()) > 0.5:
        k = tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32) # Tire un entier k dans {0,1,2,3} : nombre de quarts de tour (0°, 90°, 180°, 270°).
        img  = tf.image.rot90(img, k)
        mask = tf.image.rot90(mask, k)
    return img, mask


**Cellule — : datasets**

In [ ]:
ds_train = tf.data.Dataset.from_tensor_slices((x_train, y_train)) \
    .map(tf_load_pair, num_parallel_calls=tf.data.AUTOTUNE) \
    .map(augment,      num_parallel_calls=tf.data.AUTOTUNE) \
    .shuffle(512) \
    .batch(BATCH_SIZE) \
    .prefetch(tf.data.AUTOTUNE)

ds_val = tf.data.Dataset.from_tensor_slices((x_val, y_val)) \
    .map(tf_load_pair, num_parallel_calls=tf.data.AUTOTUNE) \
    .batch(BATCH_SIZE) \
    .prefetch(tf.data.AUTOTUNE)

# ✅ Check attendu
xb, yb = next(iter(ds_train))
print(xb.shape, yb.shape, xb.dtype, yb.dtype)


## 7) Modèle U‑Net (léger) & compile

**Cellule — : `build_unet(input_shape, base=32)`**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def enc(x,f):# encodeur
    x = layers.Conv2D(f,3,activation="relu",padding="same")(x)
    x = layers.Conv2D(f,3,activation="relu",padding="same")(x)
    p = layers.MaxPooling2D()(x)
    return x,p # Retourne s (skip = sortie avant pooling) et p (après pooling).

def bottleneck(x,f): # Deux conv au fond du U (pas de pooling)
    x = layers.Conv2D(f,3,activation="relu",padding="same")(x)
    x = layers.Conv2D(f,3,activation="relu",padding="same")(x)
    return x

# - dec: Conv2DTranspose(..., strides=2) -> Concat skip -> Conv2D x2
def dec(x,skip,f): # décodeur
    x = layers.Conv2DTranspose(f,2,strides=2,padding="same")(x)
    x = layers.Concatenate()([x,skip])
    x = layers.Conv2D(f,3,activation="relu",padding="same")(x)
    x = layers.Conv2D(f,3,activation="relu",padding="same")(x)
    return x
# - sortie: Conv2D(1,1,activation='sigmoid')

def build_unet(input_shape=(TARGET_SIZE,TARGET_SIZE,1)): # Construction du modèle complet
    inp = layers.Input(input_shape)
    s1,p1 = enc(inp,32);  s2,p2 = enc(p1,64)
    s3,p3 = enc(p2,128);  s4,p4 = enc(p3,256)
    b = bottleneck(p4,512)
    d1 = dec(b,s4,256);  d2 = dec(d1,s3,128)
    d3 = dec(d2,s2,64); d4 = dec(d3,s1,32)
    out = layers.Conv2D(1,1,activation="sigmoid")(d4)
    return keras.Model(inp,out)


model = build_unet() 
model.summary()


**Cellule — compile**

In [ ]:
#  - simple : 'binary_crossentropy'
#  - mieux (déséquilibre) : BCE + SoftDice (à ajouter plus tard)
import tensorflow as tf
from tensorflow import keras

def iou_coef(y_true, y_pred, smooth=1e-6):
    # cast
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    # soft IoU (pas de seuillage ici)
    intersection = tf.reduce_sum(y_true * y_pred, axis=[1,2,3])
    union = tf.reduce_sum(y_true + y_pred, axis=[1,2,3]) - intersection
    return tf.reduce_mean((intersection + smooth) / (union + smooth))

def dice_coef(y_true, y_pred, smooth=1e-6):
    # cast
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    # soft Dice (pas de seuillage ici)
    intersection = tf.reduce_sum(y_true * y_pred, axis=[1,2,3])
    denom = tf.reduce_sum(y_true, axis=[1,2,3]) + tf.reduce_sum(y_pred, axis=[1,2,3])
    return tf.reduce_mean((2.0 * intersection + smooth) / (denom + smooth))

bce = keras.losses.BinaryCrossentropy(from_logits=False)

def dice_loss(y_true, y_pred, smooth=1e-6):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred, axis=[1,2,3])
    denom = tf.reduce_sum(y_true, axis=[1,2,3]) + tf.reduce_sum(y_pred, axis=[1,2,3])
    # on retourne la moyenne batch
    return 1.0 - tf.reduce_mean((2.0 * intersection + smooth) / (denom + smooth))

def bce_dice_loss(y_true, y_pred):
    return bce(y_true, y_pred) + dice_loss(y_true, y_pred)

model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss=bce_dice_loss,
    metrics=[dice_coef, iou_coef]
)


**Cellule — callbacks + fit**

In [ ]:
# ⚠️ Entraînement complet (~6h) : décommentez pour relancer

#cb = [
#    keras.callbacks.EarlyStopping(monitor="val_dice_coef", mode="max", patience=7, restore_best_weights=True),
#    keras.callbacks.ModelCheckpoint("unet_dicom_best.h5", monitor="val_dice_coef", mode="max", save_best_only=True),
#    keras.callbacks.ReduceLROnPlateau(monitor="val_dice_coef", mode="max", factor=0.5, patience=3, min_lr=1e-5)
#]
# 
## history = model.fit(train_ds, validation_data=val_ds, epochs=50, callbacks=[...])
#hist = model.fit(ds_train, validation_data=ds_val, epochs=50, callbacks=cb) # dataset de validation, utilisé à la fin de chaque époque pour mesurer la performance.
#
#
## ✅ Check attendu :
#print(hist.history.keys())  # loss/val_loss, dice_coef/val_dice_coef, ...


## 8) Courbes d’entraînement

In [ ]:
# Option : ax.set_ylim(0,1) pour les métriques

metrics = ['loss', 'dice_coef', 'iou_coef']


fig, axes = plt.subplots(3,1, figsize=(8,12))  # 2 lignes × 2 colonnes
axes = axes.ravel()  #on “aplatit” ce tableau pour avoir une liste [case1, case2, case3, case4] facile à parcourir dans une boucle.
for i, m in enumerate(metrics):
    ax = axes[i]
    ax.plot(hist.history[m], label=f'Train {m}')
    ax.plot(hist.history[f'val_{m}'], label=f'Val {m}')
    ax.set_title(m)
    ax.set_xlabel('Epoch')
    ax.set_ylabel(m)
    ax.set_ylim(0,1)
    ax.grid(True)
    ax.legend()

plt.tight_layout()  # pour éviter que les titres/légendes se chevauchent
plt.show()


In [ ]:
import numpy as np, cv2, matplotlib.pyplot as plt, tensorflow as tf

# 1) Récupérer TOUTES les probas & GT du ds_val
probs_list, gts_list, imgs_list = [], [], []
for xb, yb in ds_val:
    pb = model.predict(xb, verbose=0).squeeze(-1)   # (B,256,256), renvoie un tenseur (B, H, W, 1) de proba dans [0,1] et .squeeze(-1) supprime le dernier canal singleton
    probs_list.append(pb)
    gts_list.append(yb.numpy().squeeze(-1).astype(np.uint8))
    imgs_list.append(xb.numpy())
probs = np.concatenate(probs_list, 0)
gts   = np.concatenate(gts_list,   0)
imgs  = np.concatenate(imgs_list,  0)

# 2) métriques binaires
def dice_bin(a,b):
    a = a.astype(bool); b = b.astype(bool)
    inter = np.logical_and(a,b).sum()
    return (2*inter)/(a.sum()+b.sum()+1e-7)
def iou_bin(a,b):
    a = a.astype(bool); b = b.astype(bool)
    inter = np.logical_and(a,b).sum()
    union = np.logical_or(a,b).sum()
    return inter/(union+1e-7)

# 3) plus grande composante
def keep_largest_component(mask_bin_2d):
    m = (mask_bin_2d.astype(np.uint8) > 0).astype(np.uint8)
    if m.sum() == 0: return m
    n, labels = cv2.connectedComponents(m, connectivity=8)# cv2.connectedComponents = OpenCV scanne l’image binaire et attribue un numéro différent à chaque “îlot” de pixels connectés.
    if n <= 1: return m
    sizes = [(labels==lab).sum() for lab in range(1, n)]
    largest = int(np.argmax(sizes)) + 1
    return (labels == largest).astype(np.uint8)

# 4) chercher le meilleur seuil (0.1–0.6)
ths = np.linspace(0.1, 0.6, 26)
best_t, best_d = 0.5, -1.0
for t in ths:
    d = np.mean([dice_bin(g, (p>=t).astype(np.uint8)) for g,p in zip(gts, probs)])
    if d > best_d: best_d, best_t = d, t
print(f"[val] meilleur seuil = {best_t:.2f} | Dice moyen (sans post) = {best_d:.4f}")

# 5) appliquer seuil + LCC et calculer Dice/IoU finaux
preds_bin = []
for p in probs:
    b = (p >= best_t).astype(np.uint8)
    b = keep_largest_component(b)
    preds_bin.append(b)
preds_bin = np.stack(preds_bin, 0)

dice_final = np.mean([dice_bin(g, b) for g,b in zip(gts, preds_bin)])
iou_final  = np.mean([iou_bin(g,  b) for g,b in zip(gts, preds_bin)])
print(f"[val] Dice (post) = {dice_final:.4f} | IoU (post) = {iou_final:.4f}")

# 6) afficher 3 exemples POSITIFS
pos_idx = [i for i,g in enumerate(gts) if g.sum()>0]
show = pos_idx[:3] if len(pos_idx)>=3 else list(range(min(3, len(gts))))
n = len(show)
plt.figure(figsize=(14, 4*n))
for j,i in enumerate(show):
    plt.subplot(n,4,4*j+1); plt.imshow(imgs[i,...,0], cmap='gray'); plt.title("Image"); plt.axis('off')
    plt.subplot(n,4,4*j+2); plt.imshow(gts[i], cmap='gray');         plt.title("Mask GT"); plt.axis('off')
    plt.subplot(n,4,4*j+3); plt.imshow(probs[i], cmap='gray', vmin=0, vmax=1); plt.title("Pred proba"); plt.axis('off')
    plt.subplot(n,4,4*j+4); plt.imshow(preds_bin[i], cmap='gray');   plt.title(f"Pred @{best_t:.2f} + LCC"); plt.axis('off')
plt.tight_layout(); plt.show()


## 9) Couverture (proportion de pixels positifs)

In [ ]:

def compute_coverage(ds):
    total = 0.0
    count = 0
    for _, y in ds.unbatch(): # ds_train.unbatch() → éclate chaque batch en exemples individuels (un (x,y) à la fois).
        total += y.numpy().mean() # moyenne des pixels du masque (comme c’est binaire {0,1}, ça = % de pixels positifs).
        count += 1
    return total / count

cov_train = compute_coverage(ds_train) # ~6.8% des pixels en moyenne appartiennent au foie.
cov_val   = compute_coverage(ds_val) # ~5.5% des pixels en moyenne appartiennent au foie.
print('Couverture Train :', cov_train)
print('Couverture Val   :', cov_val)
# Interprétation : ~0.00–0.02 → très rare (déséquilibre) ; ~0.05–0.15 → raisonnable


## 11) Sauvegardes pour le README

In [ ]:
import os

# 1) Créer un dossier "results" si besoin
os.makedirs("results", exist_ok=True)

# ============================================================
# PARTIE 1 : Sauvegarder quelques figures (images val + GT + prédiction)
# ============================================================
for j,i in enumerate(show):  # "show" = indices déjà choisis d'images positives
    plt.figure(figsize=(12,4))
    
    # Image CT
    plt.subplot(1,3,1)
    plt.imshow(imgs[i,...,0], cmap='gray')
    plt.title("Image"); plt.axis('off')
    
    # Masque GT
    plt.subplot(1,3,2)
    plt.imshow(gts[i], cmap='gray')
    plt.title("Mask GT"); plt.axis('off')
    
    # Overlay (image + préd)
    plt.subplot(1,3,3)
    plt.imshow(imgs[i,...,0], cmap='gray')
    plt.imshow(preds_bin[i], alpha=0.4, cmap='Reds')  # alpha=0.4 pour transparence
    plt.title(f"Pred @{best_t:.2f} + LCC"); plt.axis('off')
    
    plt.tight_layout()
    plt.savefig(f"results/example_{j}.png")  # Sauvegarde l’image
    plt.close()  # Ferme la figure pour ne pas saturer la mémoire

print("✅ Exemples sauvegardés dans results/")

# ============================================================
# PARTIE 2 : Sauvegarder les courbes d'entraînement
# ============================================================
metrics = ['loss', 'dice_coef', 'iou_coef']
fig, axes = plt.subplots(3,1, figsize=(8,12))
axes = axes.ravel()
for i, m in enumerate(metrics):
    ax = axes[i]
    ax.plot(hist.history[m], label=f'Train {m}')
    ax.plot(hist.history[f'val_{m}'], label=f'Val {m}')
    ax.set_title(m); ax.set_xlabel('Epoch'); ax.set_ylabel(m)
    ax.set_ylim(0,1); ax.grid(True); ax.legend()
plt.tight_layout()
plt.savefig("results/training_curves.png")
plt.close()
print("✅ Courbes d'entraînement sauvegardées dans results/training_curves.png")

# ============================================================
# PARTIE 3 : Sauvegarder un résumé texte
# ============================================================
with open("results/summary.txt", "w") as f:
    f.write("=== Résumé expérimentation CHAOS Foie ===\n\n")
    f.write("Patients train : " + str(sorted(train_ids)) + "\n")
    f.write("Patients val   : " + str(sorted(val_ids)) + "\n\n")
    f.write(f"Couverture train : {cov_train:.3f}\n")
    f.write(f"Couverture val   : {cov_val:.3f}\n\n")
    f.write(f"Meilleur seuil   : {best_t:.2f}\n")
    f.write(f"Dice final (val) : {dice_final:.4f}\n")
    f.write(f"IoU final (val)  : {iou_final:.4f}\n")
print("✅ Résumé sauvegardé dans results/summary.txt")


In [ ]:
with open("results/summary.txt", "r") as f:
    print(f.read())


---
## 12) Notes
- Le dataset **CHAOS** n'est pas inclus. Placez-le sous `Train_Sets/CT/<patient_id>/{DICOM_anon, Ground}`.
- Les résultats (figures, résumé) sont sauvegardés dans `results/`.
- Modèle entraîné : `unet_dicom_best.h5` (via `ModelCheckpoint`).
- Pour reproduire l'évaluation, exécutez les cellules **post-entraînement** (balayage de seuil + LCC) sans relancer `fit`.


In [2]:
import os
os.getcwd()


'/Users/sarahchraibi/Projet_seg_GitHub'

In [3]:
!ls

CHAOS_Train_Sets.zip                results
CHAOS_UNet_clean.ipynb              Train_Sets
CHAOS_UNet_TODO_Guide_patched.ipynb unet_dicom_best.h5
CHAOS_UNet_TODO_Guide.ipynb
